# Tensorflow Tutorial GAN

In [ ]:
##############
## Basic starting point from Tensorflow tutorial for DCGAN

In [ ]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import segyio
import os
import PIL
from tensorflow.keras import layers
import time

In [ ]:
## Get seismic images



# mnist example
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5 # Normalize the images to [-1, 1]

In [ ]:
# Shuffle data and create batches

buf_size = None  # should be a number equal to or greater than dataset
batch_size = None  # possibly start with 16?

train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(buf_size).batch(batch_size)

In [ ]:
"""
Model follow-ups:

This workbook persues images (i.e. spectrograms), could move to 1D convolutions and RNN / LSTM
and use trace data as input

"""

In [ ]:
# Define model architectures (initial example assumes 28x28 image)

def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256) # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 28, 28, 1)

    return model



def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[28, 28, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [ ]:
# Define loss and optimizers

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)


generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
# Checkpoint creation

checkpoint_dir = ''   # Drive directory
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
# Define training loop


seed = tf.random.normal([num_examples_to_generate, noise_dim])

@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()

        for image_batch in dataset:
            train_step(image_batch)

    # Save the model every 15 epochs
        if (epoch + 1) % 15 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))


In [ ]:
# Restore a previous checkpoint

checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
# Train the model

EPOCHS = 200
noise_dim = 100

train(train_dataset, EPOCHS)

# Garcia SREZ

In [ ]:
## Attempt the Garcia super res model using Keras

# Discriminator:  Conv2D, 64 units, stride 2
#                 batch norm
#                 relu                  

#                 repeat with 128, 256, 512 units

#                 conv2D, 512 units, stride 1
#                 batch norm, relu
#                 repeat

#                 model ends by taking mean? 

# Generator:      Residual block, 256 units
#                 Upscale (2x, nearest neighbor)
#                 batch norm
#                 relu
#                 conv2D transpose, 256 units, stride 1

#                 repeat with 128 units

#                 conv2D, 96 units, stride 1
#                 relu
#                 repeat

#                 conv2D, units=channels (color chans?)
#                 sigmoid

# Discriminator loss:  sigmoid_cross_entropy_with_logits
# Generator loss:  sigmoid_cross_entropy_with_logits

# Optimizer: Adam

In [ ]:
from tf.keras.initializers import RandomNormal
from tf.keras import layers
from tf.keras.layers import Dense, Conv2D, Conv2DTranspose, Reshape, Flatten, Dropout, BatchNormalization, LeakyReLU, ReLU
# from keras.layers.advanced_activations import LeakyReLU
from tf.keras.models import Sequential

In [ ]:
"""
Model architectures
"""

# noise_dim = 100    # taken from other examples
# weight_initializer = RandomNormal(0, 0.02)    # taken from other examples


def residual_block(y, nb_channels):

    y.add(Conv2D(nb_channels, kernel_size=(3, 3), strides=(1, 1), padding='same')
    y.add(BatchNormalization())
    y.add(LeakyReLU())

    y.add(Conv2D(nb_channels, kernel_size=(3, 3), strides=(1, 1), padding='same')
    y.add(BatchNormalization())
    y.add(LeakyReLU())

    return y

def upscale(x):
   """
   Need function to perform X (2?) upscaling
   """




def create_generator():
  generator = Sequential()
  
  units = [256, 128, 96]
  upscale_factor_list = [2, 2]

  # Res Section 1
  residual_block(generator, units[0])
  residual_block(generator, units[0])
  upscale(upscale_factor_list[0])
  generator.add(BatchNormalization())
  generator.add(ReLU())
  generator.add(Conv2DTranspose(units[0], kernel_size=(3, 3), strides=(1, 1), padding='same', use_bias=False))

  # Res Section 2
  residual_block(generator, units[1])
  residual_block(generator, units[1])
  upscale(upscale_factor_list[1])
  generator.add(BatchNormalization())
  generator.add(ReLU())
  generator.add(Conv2DTranspose(units[1], kernel_size=(3, 3), strides=(1, 1), padding='same', use_bias=False))

  # Conv Section 3
  generator.add(Conv2D(units[2], kernel_size=(3, 3), strides=(1, 1), padding='same', use_bias=False))
  generator.add(ReLU())
  generator.add(Conv2D(units[2], kernel_size=(1, 1), strides=(1, 1), padding='same', use_bias=False))
  generator.add(ReLU())

  # Sigmoid Section 4    # why sigmoid layer in generator?
  generator.add(Conv2D(units[2], kernel_size=(1, 1), strides=(1, 1), padding='same', use_bias=False))
  # generator.add(Dense(1, activation='sigmoid'))

  return generator

###############################################################

def create_descriminator():
  discriminator = Sequential()
  
  units = [256, 128, 96]

  # Conv layer 1
  discriminator.add(Conv2D(units[0], kernel_size=(3, 3), strides=(2, 2), padding='same', use_bias=False))
  discriminator.add(BatchNormalization()
  discriminator.add(ReLU())

  # Conv layer 2
  discriminator.add(Conv2D(units[1], kernel_size=(3, 3), strides=(2, 2), padding='same', use_bias=False))
  discriminator.add(BatchNormalization()
  discriminator.add(ReLU())

  # Conv layer 3
  discriminator.add(Conv2D(units[2], kernel_size=(3, 3), strides=(2, 2), padding='same', use_bias=False))
  discriminator.add(BatchNormalization()
  discriminator.add(ReLU())

  # Conv layer 4
  discriminator.add(Conv2D(units[2], kernel_size=(3, 3), strides=(1, 1), padding='same', use_bias=False))
  discriminator.add(BatchNormalization()
  discriminator.add(ReLU())

  # Conv layer 5
  discriminator.add(Conv2D(units[2], kernel_size=(1, 1), strides=(1, 1), padding='same', use_bias=False))
  discriminator.add(BatchNormalization()
  discriminator.add(ReLU())

  # Conv layer 4
  discriminator.add(Flatten())
  discriminator.add(Dense(1, activation='sigmoid', ... ))


  return discriminator

In [ ]:
generator = create_generator()
discriminator = create_descriminator()

In [ ]:
"""
Define losses and optimizers
"""

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)


generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)


In [ ]:
"""
Define training loop
"""

noise_dim = 100

@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()

        for image_batch in dataset:
            train_step(image_batch)

    # Save the model every 15 epochs
        if (epoch + 1) % 15 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

In [ ]:
# Checkpoint creation

checkpoint_dir = ''   # Drive directory
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
# Shuffle data and create batches

buf_size = None  # should be a number equal to or greater than dataset
BATCH_SIZE = 16  # possibly start with 16?

train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(buf_size).batch(batch_size)

In [ ]:
# Train the model

EPOCHS = 100

train(train_dataset, EPOCHS)

# Another example

In [ ]:
# link   https://medium.com/analytics-vidhya/implementing-a-gan-in-keras-d6c36bc6ab5f


In [ ]:
import numpy as np

from keras.initializers import RandomNormal
from keras.layers import Dense, Conv2D, Conv2DTranspose, Reshape, Flatten, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential

In [ ]:
def create_generator():
  generator = Sequential()
  
  # Starting size
  d = 4
  generator.add(Dense(d*d*256, kernel_initializer=RandomNormal(0, 0.02), input_dim=noise_dim))
  generator.add(LeakyReLU(0.2))
  # 4x4x256
  generator.add(Reshape((d, d, 256)))
  
  # 8x8x128
  generator.add(Conv2DTranspose(128, (4, 4), strides=2, padding='same', kernel_initializer=RandomNormal(0, 0.02)))
  generator.add(LeakyReLU(0.2))
  
  # 16x16*128
  generator.add(Conv2DTranspose(128, (4, 4), strides=2, padding='same', kernel_initializer=RandomNormal(0, 0.02)))
  generator.add(LeakyReLU(0.2))
  
  # 32x32x128
  generator.add(Conv2DTranspose(128, (4, 4), strides=2, padding='same', kernel_initializer=RandomNormal(0, 0.02)))
  generator.add(LeakyReLU(0.2))
  
  # 32x32x3
  generator.add(Conv2D(channels, (3, 3), padding='same', activation='tanh', kernel_initializer=RandomNormal(0, 0.02)))
  
  generator.compile(loss='binary_crossentropy', optimizer=optimizer)
  return generator

###############################################################

def create_descriminator():
  discriminator = Sequential()
  
  discriminator.add(Conv2D(64, (3, 3), padding='same', kernel_initializer=RandomNormal(0, 0.02), input_shape=(img_cols, img_rows, channels)))
  discriminator.add(LeakyReLU(0.2))
  
  discriminator.add(Conv2D(128, (3, 3), strides=2, padding='same', kernel_initializer=RandomNormal(0, 0.02)))
  discriminator.add(LeakyReLU(0.2))
  
  discriminator.add(Conv2D(128, (3, 3), strides=2, padding='same', kernel_initializer=RandomNormal(0, 0.02)))
  discriminator.add(LeakyReLU(0.2))
  
  discriminator.add(Conv2D(256, (3, 3), strides=2, padding='same', kernel_initializer=RandomNormal(0, 0.02)))
  discriminator.add(LeakyReLU(0.2))
  
  discriminator.add(Flatten())
  discriminator.add(Dropout(0.4))
  discriminator.add(Dense(1, activation='sigmoid', input_shape=(img_cols, img_rows, channels)))
  
  discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)
  return discriminator

# Cyclegan / Pix2Pix

In [ ]:
#link       https://medium.com/analytics-vidhya/transforming-the-world-into-paintings-with-cyclegan-6748c0b85632

# utilizes Unet generator with skip connections

# pix2pix tutorial (Unet generator example)  https://www.tensorflow.org/tutorials/generative/pix2pix